# CS3033/CS6405 - Data Mining - Second Assignment

### Submission

This assignment is **due on 14/04/24 at 23:59**. You should submit a single .ipnyb file with your python code and analysis electronically via Canvas.
Please note that this assignment will account for 25 Marks of your module grade.

### Declaration

By submitting this assignment. I agree to the following:

<font color="red">“I have read and understand the UCC academic policy on plagiarism, and agree to the requirements set out thereby in relation to plagiarism and referencing. I confirm that I have referenced and acknowledged properly all sources used in the preparation of this assignment.
I declare that this assignment is entirely my own work based on my personal study. I further declare that I have not engaged the services of another to either assist me in, or complete this assignment”</font>

### Objective

The Boolean satisfiability (SAT) problem consists in determining whether a Boolean formula is satisfiable or not. This problem is one of the most widely studied combinatorial problems in computer science. It is the classic NP-complete problem. Over the past number of decades, a significant amount of research work has focused on solving SAT problems with both complete and incomplete solvers.

An extended version of the problem is Model Counting (#SAT). In #SAT the solver needs to compute the number of solutions of a Boolean formula. A wide variety of solvers have been designed to tackle this problem.

In this project, we want to create an Algorithm Selection (AS) approach to Model Counting. For each #SAT instance, there is a specific solver that works better than the others, the goal of your machine learing approach is to classify it.

In AS we represent #SAT problems with a vector of 72 features with general information about the problem, e.g., number of variables, number of clauses, etc. There is no need to understand the features to be able to complete the assignment. For each instance, there is a 'label' column representing the name of the optimal solver.


The original dataset is available at:
https://github.com/andvise/DM_Assignment/blob/main/train_data.csv



## Data Preparation

In [1]:
import pandas as pd

df = pd.read_csv("https://github.com/andvise/DM_Assignment/blob/main/train_data.csv?raw=true")
df

,c,v,clauses_vars_ratio,vars_clauses_ratio,vcg_var_mean,vcg_var_coeff,vcg_var_min,vcg_var_max,vcg_var_entropy,vcg_clause_mean,...,gsat_FirstLocalMinStep_CoeffVariance,gsat_FirstLocalMinStep_Median,gsat_FirstLocalMinStep_Q.10,gsat_FirstLocalMinStep_Q.90,gsat_BestAvgImprovement_Mean,gsat_BestAvgImprovement_CoeffVariance,gsat_FirstLocalMinRatio_Mean,gsat_FirstLocalMinRatio_CoeffVariance,gsat_EstACL_Mean,label
0,681.0,238.0,2.861345,0.349486,0.011143,0.905300,0.005874,0.111601,1.880038,0.011143,...,0.210148,50.0,42.0,57.0,0.954568,0.591296,1.141656,3.197217,9.240515e+09,gpmc
1,368.0,140.0,2.628571,0.380435,0.018012,0.510753,0.005435,0.054348,1.851609,0.018012,...,0.124438,34.0,29.0,40.0,1.693036,0.244951,0.969015,0.029930,5.401642e+03,d4
2,1935.0,1920.0,1.007812,0.992248,0.001760,1.723720,0.000000,0.012403,1.280404,0.001760,...,0.066708,102.0,94.0,111.0,0.398129,0.824694,0.935730,0.092714,3.561823e+04,gpmc
3,3452.0,2821.0,1.223680,0.817207,0.000968,1.436774,0.000290,0.006083,1.192878,0.000968,...,0.053628,192.0,179.0,205.0,0.247528,0.702251,0.923327,0.026977,1.268929e+05,gpmc
4,694.0,294.0,2.360544,0.423631,0.007656,0.493513,0.002882,0.040346,1.776102,0.007656,...,0.086841,72.0,64.0,80.0,0.822829,0.209989,0.855568,0.045802,1.647598e+04,d4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1331,949.0,351.0,2.703704,0.369863,0.006701,1.151888,0.002012,0.063380,1.857919,0.006701,...,0.079108,81.0,73.0,89.0,0.781888,0.256505,0.837929,0.066291,1.643030e+04,sharpsat
1332,1450.0,608.0,2.384868,0.419310,0.004427,1.552830,0.002408,0.161349,1.659495,0.004427,...,0.062664,136.0,125.0,147.0,0.776364,0.174557,0.855907,0.031698,3.816192e+04,gpmc
1333,250.0,100.0,2.500000,0.400000,0.026000,0.555766,0.000000,0.096000,1.751533,0.026000,...,0.140708,26.0,21.0,30.0,2.093007,0.117640,1.000000,0.000000,3.168220e+03,gpmc
1334,4949.0,3422.0,1.446230,0.691453,0.000764,0.770643,0.000202,0.004445,1.835041,0.000764,...,0.032571,461.0,441.0,479.0,0.202805,0.220101,0.856952,0.016884,6.236767e+05,gpmc


In [2]:
# Label or target variable
df['label'].value_counts()

label
gpmc        921
d4          168
ganak       140
addmc        55
sharpsat     52
Name: count, dtype: int64

# Tasks

## Basic models and evaluation (5 Marks)

Using Scikit-learn, train and evaluate a k-NN classifier using 70% of the dataset from training and 30% for testing. For this part of the project, we are not interested in optimising the parameters; we just want to get an idea of the dataset.

In [9]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

X = df.drop('label', axis=1).values  # All columns except 'label'
y = df['label'].values  # target variable named 'label'

# 70% training, 30% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Create the KNN classifier with k=5 neighbors
knn = KNeighborsClassifier(n_neighbors=5)

knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

# Evaluate the model performance (e.g., accuracy score)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6733167082294265


## Robust evaluation (10 Marks)

In this section, we are interested in more rigorous techniques by implementing more sophisticated methods. Do your best to improve the k-NN classifier performances on this dataset.

For instance, you could consider:
* Hold-out and cross-validation.
* Hyper-parameter tuning.
* Feature selection.
* Feature normalisation.
* Etc.



Your report should provide concrete information about your reasoning, everything should be well-explained.

The key to geting good marks is to show that you evaluated different methods and that you correctly selected the configuration.

In [22]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold, GridSearchCV

import warnings
warnings.filterwarnings("ignore")

def calculate_correlation(feature):
    correlation_scores[feature.name] = feature.corr(target_variable_series)

def calculate_information_gain(feature):
    if feature.dtype != object:
        reshaped_feature = feature.values.reshape(-1, 1)  # Reshape feature to single column
        reshaped_target_variable = target_variable_series.values.reshape(-1, 1)  # Reshape target variable
        information_gain_scores[feature.name] = mutual_info_classif(reshaped_feature, reshaped_target_variable)
    else:
        information_gain_scores[feature.name] = 0.0  # Information gain not suitable for categorical features

def calculate_chi2(feature):
    contingency_table = pd.crosstab(features[feature.name], target_variable)
    chi2, pval, degrees_of_freedom, expected_counts = chi2_contingency(contingency_table.values)
    chi2_scores[feature.name] = chi2

# Separate features and target variable
features = df.drop("label", axis=1)
target_variable = df["label"]

# Label Encoding (if target variable represents categories)
if target_variable.dtype == object:
    le = LabelEncoder()
    target_variable_encoded = le.fit_transform(target_variable)
    target_variable_series = pd.Series(target_variable_encoded)

# Feature Importance Scores (Dictionaries)
correlation_scores = {}
information_gain_scores = {}
chi2_scores = {}  # For categorical features

categorical_features = features.select_dtypes(include=[object])

# Calculate scores for all features
for col in features.columns:
    calculate_correlation(features[col])
    calculate_information_gain(features[col])
    if col in features.columns:
        calculate_chi2(features[col])

combined_scores = {}
for k in correlation_scores.items():
  combined_scores[k[0]] = correlation_scores[k[0]] + information_gain_scores[k[0]]

max_score =max([score[1][0] for score in combined_scores.items()])
values = np.arange(0, max_score - 0.005, 0.04)

output = []

for i in values:
  # Threshold  number_of_features_above_threshold
  # 0.3  13
  # 0.25  30
  # 0.2  46
  # 0.15 52
  # 0.1 59
  # 0.05 67
  threshold = i
  important_features = [f for f, score in combined_scores.items() if score >= threshold]

  # normalise the features
  # Create a MinMaxScaler object
  scaler = MinMaxScaler(feature_range=(0, 1))

  # Fit the scaler on the training data
  X_scaled = scaler.fit_transform(df[important_features])
  out = [threshold,len(important_features)]
  X = X_scaled
  y = df['label']

  # Define the parameter grid for hyperparameter tuning
  param_grid = {
    'n_neighbors': [1, 3, 5, 7, 9],
    'metric': ['euclidean', 'manhattan', 'minkowski']
  }

  # Define the KNN model
  knn = KNeighborsClassifier()
  cv = KFold(n_splits=10, shuffle=True, random_state=42)
  cv_results = []

  # Create the GridSearchCV object
  grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=cv, scoring='accuracy')

  # Fit the grid search to the data
  grid_search.fit(X, y)

  if isinstance(grid_search.cv_results_, dict):
    cv_results = grid_search.cv_results_['mean_test_score']
  else:
    for result in grid_search.cv_results_:
      params = result['params']
      mean_test_score = result['mean_test_score'][0]  # Assuming single scoring metric
      cv_results.append({'params': params, 'mean_test_score': mean_test_score})

  out.append(cv_results.max())
  out.append(cv_results)
  output.append(out)

print(f'\n\nAccuracy = {max([f[2] for f in output])}\n\n') # 62 parameters, kFold = 10, k = 5 with euclidean distance metric got this result.



Accuracy = 0.7657558074290203




Process:
1) Separated features and target variable
2) Label Encoding for the target variable so that values are converted into numbers which will be easier for algorithms to process.
3) Feature Selection: The dataset has many features i.e, 72 where all of them may not hold good info about the target variable so decided to cut down the features.
i) Chose Feature selection filter methods i.e, correlation analysis, information_gain_scores and chi square test. These filter scores can provide insights into the relationship between features and the target variable.
ii) Calculated the scores for all the 3 filters and calculated the final score by adding them all up by giving equal weights.
iii) list of threshold value and number of features with scores above threshold are:
 0.3  13,
  0.25  30,
  0.2  46,
  0.15 52,
  0.1 59,
 0.05 67,
4) Now employed GridSearchCV along with k fold cross validation and hyperparameter tuning. Tuned the k value and distance calculation metric of KNN classifier.
5) Many combinations are tested in the code those are:
--> Number of features selected which can changed by changing the threshold of filter scores.
--> k value of KNN Classifier i.e, 1,3,5,7,9
--> Distance metric for KNN i.e, 'euclidean', 'manhattan', 'minkowski'


Best Combination:
For threshold of 0.08, 62 features have scores above threshold.
Used k = 10 fold for cross validation.
For KNN classifier,best parameters came out as k=5 with euclidean distance metric

Out of all the above methods and different combinations achieved a 76.58% accuracy which is more than the KNN Algorithm accuracy of 67.33%.

## New classifier (10 Marks)

Replicate the previous task for a classifier different than K-NN and decision trees. Briefly describe your choice.
Try to create the best model for the given dataset.






In [33]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.feature_selection import mutual_info_classif
from scipy.stats import chi2_contingency
import warnings
warnings.filterwarnings("ignore")

def calculate_correlation(feature):
    correlation_scores[feature.name] = feature.corr(target_variable_series)

def calculate_information_gain(feature):
    if feature.dtype != object:
        reshaped_feature = feature.values.reshape(-1, 1)
        reshaped_target_variable = target_variable_series.values.reshape(-1, 1)
        information_gain_scores[feature.name] = mutual_info_classif(reshaped_feature, reshaped_target_variable)
    else:
        information_gain_scores[feature.name] = 0.0

def calculate_chi2(feature):
    contingency_table = pd.crosstab(features[feature.name], target_variable)
    chi2, _, _, _ = chi2_contingency(contingency_table.values)
    chi2_scores[feature.name] = chi2

# Load dataset
df = pd.read_csv("https://github.com/andvise/DM_Assignment/blob/main/train_data.csv?raw=true.csv")

# Separate features and target variable
features = df.drop("label", axis=1)
target_variable = df["label"]

# Label Encoding (if target variable represents categories)
if target_variable.dtype == object:
    le = LabelEncoder()
    target_variable_encoded = le.fit_transform(target_variable)
    target_variable_series = pd.Series(target_variable_encoded)

# Feature Importance Scores (Dictionaries)
correlation_scores = {}
information_gain_scores = {}
chi2_scores = {}

# Calculate scores for all features
for col in features.columns:
    calculate_correlation(features[col])
    calculate_information_gain(features[col])
    if col in features.columns:
        calculate_chi2(features[col])

# Combine scores
combined_scores = {k: correlation_scores[k] + information_gain_scores[k] for k in correlation_scores}

# Get max score for threshold
max_score = max([score[0] for score in combined_scores.values()])
values = np.arange(0, max_score - 0.005, 0.04)

output = []

# Preprocess features
scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler.fit_transform(features)

for i in values:
    threshold = i
    important_features = [f for f, score in combined_scores.items() if score >= threshold]
    X = X_scaled[:, [features.columns.get_loc(f) for f in important_features]]
    y = target_variable

    # Define the Random Forest model with hyperparameters
    rf = RandomForestClassifier(n_estimators=100, random_state=42)

    # Define the KFold cross-validation
    cv = KFold(n_splits=5, shuffle=True, random_state=42)

    # Create the GridSearchCV object with hyperparameter tuning
    grid_search = GridSearchCV(estimator=rf, param_grid={}, cv=cv, scoring='accuracy')

    # Fit the grid search to the data
    grid_search.fit(X, y)

    # Get the best CV results
    best_accuracy = grid_search.best_score_
    output.append([threshold, len(important_features), best_accuracy, grid_search.cv_results_['mean_test_score']])

best_accuracy = max([f[2] for f in output])
print(f'\n\nBest Accuracy = {best_accuracy}\n\n')



Best Accuracy = 0.7866929397954049




Other than K-nn, tried a couple more classifiers like Naive baye's and Gradient boosting.
Acheived more accuracy in **random forest**
--> Gradient Boosting took more time since the number of trees are more and even the iterations are large.
-->There might be some complex relationships in the dataset, hence got a low accuracy when Naive baye's used.
-->The features are relatively more in the dataset.
Hence, **Random forest** worked and acheived a better accuracy than the other 2 classifiers.

# <font color="blue">Testing part - Not needed for CS3033</font>

Save your best model into your GitHub. And create a single code cell that loads it and evaluates it on the following test dataset:
https://github.com/andvise/DM_Assignment/blob/main/test_data.csv

We will cover this part in a Lab session.

I should be able to run the code cell independently, load all the libraries you need as well.

This link currently contains a sample of the training set. The real test set will be released after the submission.

In [ ]:
from joblib import dump, load
from io import BytesIO
import requests
import pandas as pd

# INSERT YOUR MODEL'S URL
mLink = 'url'
mfile = BytesIO(requests.get(mLink).content)
model = load(mfile)

df = pd.read_csv("https://github.com/andvise/DataAnalyticsDatasets/blob/main/test_dataset.csv")

# Your code here